In [1]:
import numpy as np 
import pandas as pd
import requests 
import xlsxwriter 
import math 
from scipy import stats 

In [2]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={"Tpk_059b97af715d417d9f49f50b51b1c448"}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 75395347,
 'calculationPrice': 'close',
 'change': -1.19,
 'changePercent': -0.0077,
 'close': 158.89,
 'closeSource': 'acfifiol',
 'closeTime': 1715442065000,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': 154.27,
 'delayedPriceTime': 1718090108017,
 'extendedChange': -0.37,
 'extendedChangePercent': -0.00229,
 'extendedPrice': 157.7,
 'extendedPriceTime': 1707375251013,
 'high': 160.3,
 'highSource': 'latdyeucpe  ii1emne rd5',
 'highTime': 1720804324254,
 'iexAskPrice': 0,
 'iexAskSize': 0,
 'iexBidPrice': 0,
 'iexBidSize': 0,
 'iexClose': 155.69,
 'iexCloseTime': 1674938225354,
 'iexLastUpdated': 1719220162576,
 'iexMarketPercent': 0.01802856158033531,
 'iexOpen': 160.63,
 'iexOpenTime': 1679392723787,
 'iexRealtimePrice': 155.73,
 'iexRealtimeSize': 405,
 'iexVolume': 935366,
 'lastTradeTime': 1741036058777,
 'latestPrice': 155.68,
 'latestSource': 'Close',
 'latestTime': 'July 25, 2022',
 'latestUpdate': 1709623161254,
 'latestVolume': 5407188

In [3]:
pe_ratio = data['peRatio']
pe_ratio

25.78

In [4]:
stocks = pd.read_csv('sp_500_stocks.csv')

In [5]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))


my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={"Tpk_059b97af715d417d9f49f50b51b1c448"}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['peRatio'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

KeyError: 'DISCA'

In [7]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)

In [8]:
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,AMZN,124.84,3.0,N/A
1,COF,115.38,4.4,N/A
2,AIG,53.14,4.62,N/A
3,APA,34.73,5.19,N/A
4,DHI,78.72,5.6,N/A
5,DFS,104.62,6.17,N/A
6,C,53.40,6.2,N/A
7,CE,116.51,6.2,N/A
8,BEN,25.89,6.86,N/A
9,AIZ,180.25,7.61,N/A


In [9]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

In [10]:
portfolio_input()

Enter the value of your portfolio:100000


In [11]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,AMZN,124.84,3.0,16
1,COF,115.38,4.4,17
2,AIG,53.14,4.62,37
3,APA,34.73,5.19,57
4,DHI,78.72,5.6,25
5,DFS,104.62,6.17,19
6,C,53.40,6.2,37
7,CE,116.51,6.2,17
8,BEN,25.89,6.86,77
9,AIZ,180.25,7.61,11


In [12]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={"Tpk_059b97af715d417d9f49f50b51b1c448"}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

In [13]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={"Tpk_059b97af715d417d9f49f50b51b1c448"}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )

KeyError: 'DISCA'

In [14]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
28,ALXN,191.30,N/A,61.21,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,289.64,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,289.92,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
88,CERN,98.16,N/A,48.22,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.00,N/A,9.88,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


In [15]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

In [16]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


In [17]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_dataframe[metric])

#Print the entire DataFrame    
rv_dataframe

0      0.733333
1      0.051852
2      0.518519
3      0.637037
4      0.540741
         ...   
130    0.648148
131    0.162963
132    0.103704
133    0.762963
134    0.948148
Name: PE Percentile, Length: 135, dtype: object
0      0.785185
1      0.037037
2      0.577778
3      0.962963
4      0.903704
         ...   
130     0.82963
131    0.362963
132    0.185185
133    0.637037
134    0.296296
Name: PB Percentile, Length: 135, dtype: object
0      0.811111
1      0.022222
2      0.148148
3      0.859259
4      0.740741
         ...   
130    0.303704
131    0.237037
132    0.125926
133    0.866667
134    0.433333
Name: PS Percentile, Length: 135, dtype: object
0      0.785185
1           1.0
2      0.392593
3      0.748148
4      0.325926
         ...   
130    0.585185
131    0.185185
132    0.088889
133     0.77037
134    0.733333
Name: EV/EBITDA Percentile, Length: 135, dtype: object
0      0.814815
1      0.044444
2      0.177778
3      0.933333
4      0.637037
         ...   
1

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,125.91,N/A,30.39,0.733333,7.45,0.785185,5.8000,0.811111,21.613536,0.785185,11.065710,0.814815,N/A
1,AAL,14.36,N/A,-4.78,0.051852,-1.09,0.037037,0.2222,0.022222,105.716807,1.0,0.817119,0.044444,N/A
2,AAP,198.99,N/A,21.13,0.518519,4.05,0.577778,1.0800,0.148148,11.857600,0.392593,2.516389,0.177778,N/A
3,AAPL,158.89,N/A,26.01,0.637037,39.00,0.962963,6.7400,0.859259,20.181623,0.748148,15.371789,0.933333,N/A
4,ABBV,150.40,N/A,21.92,0.540741,16.68,0.903704,4.8300,0.740741,11.040198,0.325926,8.306357,0.637037,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,DG,255.91,N/A,26.18,0.648148,9.53,0.82963,1.6900,0.303704,15.950807,0.585185,5.722384,0.377778,N/A
131,DGX,135.13,N/A,8.77,0.162963,2.45,0.362963,1.4700,0.237037,7.704042,0.185185,4.480388,0.288889,N/A
132,DHI,76.67,N/A,5.50,0.103704,1.62,0.185185,0.9306,0.125926,4.720370,0.088889,3.271231,0.222222,N/A
133,DHR,283.81,N/A,33.51,0.762963,4.64,0.637037,6.7600,0.866667,21.277786,0.77037,11.251339,0.822222,N/A


In [18]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,125.91,N/A,30.39,0.733333,7.45,0.785185,5.8000,0.811111,21.613536,0.785185,11.065710,0.814815,0.785926
1,AAL,14.36,N/A,-4.78,0.051852,-1.09,0.037037,0.2222,0.022222,105.716807,1.0,0.817119,0.044444,0.231111
2,AAP,198.99,N/A,21.13,0.518519,4.05,0.577778,1.0800,0.148148,11.857600,0.392593,2.516389,0.177778,0.362963
3,AAPL,158.89,N/A,26.01,0.637037,39.00,0.962963,6.7400,0.859259,20.181623,0.748148,15.371789,0.933333,0.828148
4,ABBV,150.40,N/A,21.92,0.540741,16.68,0.903704,4.8300,0.740741,11.040198,0.325926,8.306357,0.637037,0.62963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,DG,255.91,N/A,26.18,0.648148,9.53,0.82963,1.6900,0.303704,15.950807,0.585185,5.722384,0.377778,0.548889
131,DGX,135.13,N/A,8.77,0.162963,2.45,0.362963,1.4700,0.237037,7.704042,0.185185,4.480388,0.288889,0.247407
132,DHI,76.67,N/A,5.50,0.103704,1.62,0.185185,0.9306,0.125926,4.720370,0.088889,3.271231,0.222222,0.145185
133,DHR,283.81,N/A,33.51,0.762963,4.64,0.637037,6.7600,0.866667,21.277786,0.77037,11.251339,0.822222,0.771852


In [19]:
rv_dataframe.sort_values(by = 'RV Score', inplace = True)
rv_dataframe = rv_dataframe[:10]
rv_dataframe.reset_index(drop = True, inplace = True)

In [20]:
portfolio_input()

Enter the value of your portfolio:10000


In [21]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])-1):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

C:\Users\swara\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AIG,54.35,18,4.65,0.088889,0.7576,0.051852,0.7904,0.103704,2.703239,0.037037,0.776088,0.02963,0.062222
1,APA,35.29,28,5.36,0.096296,-626.6400,0.007407,1.2600,0.181481,3.347396,0.059259,1.818680,0.111111,0.091111
2,COF,111.01,9,4.43,0.081481,0.7645,0.059259,1.3000,0.2,2.941870,0.044444,1.293811,0.081481,0.093333
3,CAH,57.63,17,-17.60,0.044444,-22.9500,0.014815,0.0880,0.007407,8.963007,0.22963,2.952079,0.2,0.099259
4,ALL,121.78,8,9.80,0.2,1.5800,0.162963,0.6357,0.088889,3.033708,0.051852,0.628072,0.014815,0.103704
5,C,54.30,18,6.38,0.125926,0.5836,0.044444,1.3100,0.207407,3.991540,0.074074,1.297905,0.088889,0.108148
6,AIZ,175.19,5,7.52,0.140741,1.8400,0.233333,0.9182,0.118519,2.577263,0.02963,0.880803,0.059259,0.116296
7,BEN,25.24,39,6.81,0.133333,1.1000,0.081481,1.5200,0.244444,4.818037,0.096296,1.268803,0.074074,0.125926
8,DHI,76.67,13,5.50,0.103704,1.6200,0.185185,0.9306,0.125926,4.720370,0.088889,3.271231,0.222222,0.145185
9,ALK,44.90,N/A,12.24,0.259259,1.5200,0.148148,0.7758,0.096296,8.531270,0.207407,0.692208,0.022222,0.146667


In [31]:
rv_dataframe.to_csv("ValueInvesting_Stocks.csv")